# Section 1 

In [1]:
from __future__ import division

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import numpy as np

import time

# Section 2

##### Public Static Final Variables

In [1]:
filename='GBPUSD1d.txt'
path='' # Blank := local file
filelocation=path+filename

comma=','
patternFinderStoppingPoint=60
ptrnStart=31 #patternFinderStartingPoint

similarityThreshold=20
#1e-31 which is clsoe to zero, but will not introduce division by zero error
almostZero=0.0000000000000000000000000000001
almostZero=0.000001


##### Global Variables

In [2]:
date,bid,ask = np.loadtxt(filelocation, unpack=True, 
                      delimiter=comma, 
                      converters={0:mdates.strpdate2num('%Y%m%d%H%M%S')})
                    #converters: Used to convert DateStamps to plot Dates

avgLine = (bid+ask)/2

#Store the Pattern and store the outcome of the next 20-30 points 
patternAr = []
performanceAr = []

patForRec = []

NameError: name 'np' is not defined

# Section 3

In [3]:
def graphRawFX():
    global date,bid,ask
    
    fig = plt.figure(figsize=(10,7))
    ax1 = plt.subplot2grid((40,40), (0,0), rowspan=40, colspan=40)

    ax1.plot(date,bid)
    ax1.plot(date,ask)
    plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)


    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    for label in ax1.xaxis.get_ticklabels():
        label.set_rotation(45)

    # Spread addition to the plot
    ax1_2 = ax1.twinx()
    ax1_2.fill_between(date, 0, (ask-bid), facecolor='g', alpha=.3)

    plt.subplots_adjust(bottom=.23)

    plt.grid(True)
    plt.show()

In [4]:
#graphRawFX()

# Section 4

In [5]:
def percentChange(startPoint_, currentPoint_):
    global almostZero
    # We use try to make sure we do not get a division by zero error
    # We return 0.000...001 instead of zero to avoid similar issues.
    try:
        perChange = (float(currentPoint_-startPoint_)*100.0/abs(startPoint_))
        if perChange == 0.0:
            return almostZero
        else:
            return perChange
    except:
        return almostZero
    return almostZero

# Section 5

In [6]:
def patternStorage():
    global avgLine, ptrnStart, patternFinderStartingPoint
    
    #Timing it
    patStartTime = time.time()
    
    # counter end point; -60 to stop at the 60th point
    x = len(avgLine) - patternFinderStoppingPoint 
    
    y = ptrnStart#counter starting point 
    while y < x:
        pattern = []
        #p1=point 1; avgLine[1]=startingpoint; avgLine[1:ptrnEnd]=currentpoint
        #Repeat the process for every starting point and subtract 1 from "ptrnStart" per point.
        for counter in xrange(1,ptrnStart):
            pattern.append(percentChange(avgLine[1], avgLine[1+counter]))
        
        outcomeRange = avgLine[y+20:y+30] # TODO!!! This needs changing!!!!!
        currentPoint = avgLine[y]
        
        try:
            avgOutcome =(reduce(lambda x,y: x+y, outcomeRange)/len(outcomeRange))
        except Exception, e:
            print str(e)
            avgOutcome=0
        
        
        #Normalise 
        futureOutcome = percentChange(currentPoint, avgOutcome)
        
        patternAr.append(pattern)
        performanceAr.append(futureOutcome)
        
        y += 1
        
    patEndTime = time.time() - patStartTime
    
    #Sanity check lengths are the same 
    print('Length of pattern Array: {0}'.format(len(patternAr)))
    print('Length of performance Array: {0}'.format(len(performanceAr)))
    
    print('Pattern storage took: {0}{1}'.format(patEndTime, ' seconds'))
    
                

In [8]:
#patternStorage()

# Section 7

In [9]:
def currentPattern():
    global ptrnStart, patForRec
    #pattern for Recognition -> patForRec
    
    #Current Pattern
    for counter in xrange(ptrnStart,1,-1): 
        patForRec.append(percentChange(avgLine[-ptrnStart], avgLine[1-counter]))
    
    print("Current Pattern for recognition:\n{0}".format(patForRec)) 
    

In [10]:
#currentPattern()

# Section 8

In [11]:
def patternRecognition():

    # for every pattern in our array, measure it with the pattern in question
    # and compute their similarities. 
    for eachPattern in patternAr:
        howSim = 0
        for timept in xrange(0,(ptrnStart-1)):
            howSim+=abs(100.00-abs(percentChange(eachPattern[timept], patForRec[timept])))
            ''' print(howSim)
            print("How similar are the 2: ", eachPattern[timept], patForRec[timept])
            print("output of percent change function: ", percentChange(eachPattern[timept], patForRec[timept]))
            print("output from 100 - ans: ", 100.00-abs(percentChange(eachPattern[timept], patForRec[timept])))
            time.sleep(5)'''
            
        # How similar are we ? 
        howSim = howSim*1.0/(ptrnStart-1)
        print("Similarity is: {0}%".format(howSim))
        
        if(howSim > similarityThreshold):
            
            #Pattern Index that we found similar.
            patdex = patternAr.index(eachPattern)
            
#             print('#################################')
#             print('Pattern for Recognition:{0}{1}'.format("\n",patForRec))
            print('=================================')
            print('Potential pattern found: {0}'.format(eachPattern))
            print('With Similarity: {0}{1}{2}'.format(howSim, ' and index: ', patdex))
#             print('--------------------------------')
#             print('Predicted outcome:{0}{1}'.format("\n",performanceAr[patdex]))
            
            ##### Plotting #####
            #x axes represents the points in time (e.g. time point 1,2,3,... etc.)
            xp = range(1,31)
            #figure
            fig = plt.figure()
            plt.plot(xp, patForRec)
            plt.plot(xp, eachPattern)
            plt.show()
            ##### Plotting #####'''
            
            print('#################################')
            


## Execution 

In [12]:
patternStorage()

Length of pattern Array: 61921
Length of performance Array: 61921
Pattern storage took: 2.55900001526 seconds


In [ ]:
currentPattern()

Current Pattern for recognition:
[1e-11, -0.00032112109797936193, 1e-11, 1e-11, 1e-11, 1e-11, 1e-11, 1e-11, 1e-11, -0.00064224219595872387, 1e-11, 0.00064224219595872387, 0.00064224219595872387, 0.00096336329392382512, 0.00064224219595872387, 0.00064224219595872387, 0.00032112109797936193, 0.00032112109797936193, -0.00032112109797936193, -0.00032112109797936193, -0.00064224219595872387, -0.001605605489882549, -0.00096336329393808574, -0.00064224219595872387, -0.00096336329393808574, -0.0019267265878761715, -0.001926726587861911, -0.001926726587861911, -0.00064224219595872387, -0.00064224219595872387]


In [ ]:
patternRecognition()

Similarity is: 214080742.453%
Potential pattern found: [-0.0019309616832820227, -0.0032182694721319405, -0.0032182694721319405, -0.0035400964193622849, -0.0038619233665640454, -0.005471058102644308, -0.0061147119970621205, -0.0045055772609961502, -0.0045055772609961502, -0.0048274042082122031, -0.0048274042082122031, -0.0064365389442924649, -0.0064365389442924649, -0.00579288504986036, -0.00579288504986036, -0.0061147119970621205, -0.0070801928387102782, -0.0067583658914942253, -0.0064365389442924649, -0.0061147119970621205, -0.0051492311554282551, -0.0051492311554282551, -0.0045055772609961502, -0.0048274042082122031, -0.0048274042082122031, -0.0070801928387102782, -0.0070801928387102782, -0.0032182694721462324, -0.00032182694721605244, 1e-11]
With Similarity: 214080742.453 and index: 0
#################################
Similarity is: 214080742.453%
Potential pattern found: [-0.0019309616832820227, -0.0032182694721319405, -0.0032182694721319405, -0.0035400964193622849, -0.003861923366

In [36]:
#totalEndTime = time.time() - totalStartTime
#print('Entire Process took ', totalEndTime)

In [14]:
#Time the whole Script
totalStartTime = time.time()
patternStorage()
currentPattern()
patternRecognition()
totalEndTime = time.time() - totalStartTime
print('Entire Process took ', totalEndTime)

Length of pattern Array: 123842
Length of performance Array: 123842
Pattern storage took: 2.28500008583 seconds
Current Pattern for recognition:
[1e-31, -0.00032112109797936193, 1e-31, 1e-31, 1e-31, 1e-31, 1e-31, 1e-31, 1e-31, -0.00064224219595872387, 1e-31, 0.00064224219595872387, 0.00064224219595872387, 0.00096336329392382512, 0.00064224219595872387, 0.00064224219595872387, 0.00032112109797936193, 0.00032112109797936193, -0.00032112109797936193, -0.00032112109797936193, -0.00064224219595872387, -0.001605605489882549, -0.00096336329393808574, -0.00064224219595872387, -0.00096336329393808574, -0.0019267265878761715, -0.001926726587861911, -0.001926726587861911, -0.00064224219595872387, -0.00064224219595872387, 1e-31, -0.00032112109797936193, 1e-31, 1e-31, 1e-31, 1e-31, 1e-31, 1e-31, 1e-31, -0.00064224219595872387, 1e-31, 0.00064224219595872387, 0.00064224219595872387, 0.00096336329392382512, 0.00064224219595872387, 0.00064224219595872387, 0.00032112109797936193, 0.00032112109797936193,

KeyboardInterrupt: 

In [ ]:
0.0
('How similar are the 2: ', -0.0019309616832820227, 1e-31)
('output of percent change function: ', 100.0)
('output from 100 - ans: ', 0.0)
9.9780674291
('How similar are the 2: ', -0.0032182694721319405, -0.00032112109797936193)
('output of percent change function: ', 90.021932570903232)
('output from 100 - ans: ', 9.9780674290967681)
9.9780674291
('How similar are the 2: ', -0.0032182694721319405, 1e-31)
('output of percent change function: ', 100.0)
('output from 100 - ans: ', 0.0)

In [17]:
percentChange(-0.0019309616832820227, 1e-31)

100.0

In [18]:
percentChange(-0.0032182694721319405, -0.00032112109797936193)

90.02193257090323